## Part 1: Preprocessing

In [552]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [553]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [554]:
# Create y_df with the Attrition and Department columns
# y_attrition_df = attrition_df["Attrition"]
# y_department_df = attrition_df["Department"]

In [555]:
# Create a list of at least 10 column names to use as X data
sel_columns = ["Age", "DistanceFromHome", "Education", "HourlyRate", "JobSatisfaction",\
              "NumCompaniesWorked", "PercentSalaryHike", "PerformanceRating", "WorkLifeBalance", "YearsSinceLastPromotion"]


# Create X_df using your selected columns
X_df = attrition_df[sel_columns]

# Show the data types for X_df
X_df.dtypes


Age                        int64
DistanceFromHome           int64
Education                  int64
HourlyRate                 int64
JobSatisfaction            int64
NumCompaniesWorked         int64
PercentSalaryHike          int64
PerformanceRating          int64
WorkLifeBalance            int64
YearsSinceLastPromotion    int64
dtype: object

In [556]:
# Create y_df with the Attrition and Department columns
y_attrition_df = attrition_df["Attrition"]
y_department_df = attrition_df["Department"]

In [557]:
# Create a OneHotEncoder for the Attrition column
y_attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
# y_attrition_encoded = y_attrition_encoder.fit_transform(y_attrition_df["Attrition"])
y_attrition_encoded = y_attrition_encoder.fit_transform(y_attrition_df.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data

y_attrition_encoded


array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [558]:
# Create a OneHotEncoder for the Department column
y_department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_department_encoded = y_department_encoder.fit_transform(y_department_df.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_encoded 



array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [559]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_attr_train, y_attr_test, y_dept_train, y_dept_test = train_test_split(X_df, y_attrition_encoded, y_department_encoded)

In [560]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


In [561]:
# Create a StandardScaler
scaler = StandardScaler() #.fit(X_train)

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [562]:
# Create a OneHotEncoder for the Department column


# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data




In [563]:
# Create a OneHotEncoder for the Attrition column


# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data



## Create, Compile, and Train the Model

In [564]:
X_train_scaled.shape

(1102, 10)

In [565]:
X_test_scaled.shape

(368, 10)

In [566]:
y_attr_train.shape

(1102, 2)

In [567]:
y_attr_test.shape

(368, 2)

In [568]:
y_dept_train.shape

(1102, 3)

In [569]:
y_dept_test.shape

(368, 3)

In [570]:
# Find the number of columns in the X training data
column_nums = (X_train.shape[1],)

# Create the input layer
input_layer = layers.Input(shape=column_nums, name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [571]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden = layers.Dense(16, activation='relu', name='attr_hidden')(shared_layer2)

# Create the output layer
attr_output = layers.Dense(2, activation='softmax',name='attr_output')(attr_hidden)


In [572]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(16, activation='relu', name='dept_hidden')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(3, activation='softmax', name='dept_output')(dept_hidden)


In [573]:
# Create the model
# model = Model(inputs=input_layer, outputs=[attr_output, dept_output])
model = Model(inputs=input_layer, 
              outputs=[attr_output, dept_output],
              name='predict_attrition_model')

# Compile the model
# model.compile(optimizer='adam',
#               loss={'attribute_output': 'categorical_crossentropy', 'department_output': 'categorical_crossentropy'},
#               metrics={'attribute_output': 'accuracy', 'department_output': 'accuracy'})

model.compile(optimizer='adam',
              loss={'attr_output': 'categorical_crossentropy',
                    'dept_output': 'categorical_crossentropy'},
              loss_weights={'attr_output': 1.0, 'dept_output': 1.0, },
              metrics={'attr_output': 'accuracy',
                       'dept_output': 'accuracy'}
              )


# Summarize the model
model.summary()

Model: "predict_attrition_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 64)        │        704 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 32)        │      2,080 │ dense_36[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attr_hidden (Dense) │ (None, 16)        │        528 │ dense_37[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_hidden (Dense) │ (None, 16)        │        528 │ dense_37[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attr_output (Dense) │ (None, 2)         │         34 │ attr_hidden[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         51 │ dept_hidden[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
model.fit(
    X_train_scaled,
    {'attr_output': y_attr_train, 'dept_output': y_dept_train},
    epochs=15,
    batch_size=5,
    validation_split=0.2
)


# history = model.fit(
#     X_train, 
#     {'dept_output': y_dept_train, 'attr_output': y_attr_train},
#     validation_data=(X_test, {'dept_output': y_dept_test, 'attr_output': y_attr_test}),
#     epochs=50,
#     batch_size=32,
#     verbose=1
# )

Epoch 1/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - attr_output_accuracy: 0.7613 - dept_output_accuracy: 0.5452 - loss: 1.4723 - val_attr_output_accuracy: 0.8235 - val_dept_output_accuracy: 0.6471 - val_loss: 1.3221
Epoch 2/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - attr_output_accuracy: 0.8465 - dept_output_accuracy: 0.6388 - loss: 1.2061 - val_attr_output_accuracy: 0.8235 - val_dept_output_accuracy: 0.6425 - val_loss: 1.3161
Epoch 3/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - attr_output_accuracy: 0.8487 - dept_output_accuracy: 0.6458 - loss: 1.1707 - val_attr_output_accuracy: 0.8281 - val_dept_output_accuracy: 0.6471 - val_loss: 1.3135
Epoch 4/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - attr_output_accuracy: 0.8469 - dept_output_accuracy: 0.6334 - loss: 1.1535 - val_attr_output_accuracy: 0.8235 - val_dept_output_accuracy: 0.6425 - val_loss: 1.3272
Epoch 5/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - attr_output_accuracy: 0.8405 - dept_output_accuracy: 0.6224 - loss: 1

In [ ]:
# # Evaluate the model with the testing data
# test_results = model.evaluate(X_test_scaled, 
#                               {'attr_output': y_attr_test, 'dept_output': y_dept_test},
#                               verbose=1)

# # Print the results
# print("Test Loss:", test_results[0])
# # print("Attrition Output Loss:", test_results[1])
# # print("Department Output Loss:", test_results[2])
# print("Attrition Output Accuracy:", test_results[1])
# print("Department Output Accuracy:", test_results[2])

In [ ]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'attr_output': y_attr_test, 'dept_output': y_dept_test})
test_results

In [ ]:
# Print the accuracy for both department and attrition
print(f"Attrition Accuracy: {test_results[1]}")
print(f"Department Accuracy: {test_results[2]}")

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 